In [3]:
import numpy as np
import icecream as ic
from tqdm.auto import tqdm

In [33]:
# Problem 0:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [24]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [4]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [35]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [5]:
def is_valid(solution):
    if not np.all(solution.sum(axis=0) <= 1):
        return False

    for k in range(NUM_KNAPSACKS):
        items_in_knapsack_k = solution[k]
        weight_of_knapsack_k = WEIGHTS[items_in_knapsack_k].sum(axis=0)
        if not np.all(weight_of_knapsack_k <= CONSTRAINTS):
            return False

    return True

In [6]:
def evaluate(solution):
    if not is_valid(solution):
        return -1.0
    items_placed = np.any(solution, axis=0)
    total_value = VALUES[items_placed].sum()
    return float(total_value)

In [7]:
def move(solution):
    neighbor = solution.copy()
    item_to_move = rng.integers(0, NUM_ITEMS)
    new_knapsack_idx = rng.integers(-1, NUM_KNAPSACKS)
    neighbor[:, item_to_move] = False
    if new_knapsack_idx != -1:
        neighbor[new_knapsack_idx, item_to_move] = True

    return neighbor

In [69]:
def hill_climber_fast(initial_solution: np.ndarray, max_steps_no_improvement: int):
    current_solution, current_score = initial_solution, evaluate(initial_solution)
    steps_without_improvement = 0
    while steps_without_improvement < max_steps_no_improvement:
        neighbor = move(current_solution)
        neighbor_score = evaluate(neighbor)
        if neighbor_score > current_score:
            current_solution, current_score = neighbor, neighbor_score
            steps_without_improvement = 0
        else:
            steps_without_improvement += 1
    return current_solution, current_score

In [51]:
def create_random_valid_solution():
    
    solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    shuffled_items = list(range(NUM_ITEMS))
    rng.shuffle(shuffled_items)
    for item_idx in shuffled_items:
        knapsack_idx = rng.integers(0, NUM_KNAPSACKS)
        solution[knapsack_idx, item_idx] = True
        if not is_valid(solution):
            solution[knapsack_idx, item_idx] = False
            
    return solution

In [9]:
def crossover(parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
    child = np.zeros_like(parent1)
    for i in range(NUM_ITEMS):
        if rng.random() < 0.5:
            child[:, i] = parent1[:, i]
        else:
            child[:, i] = parent2[:, i]
    return child

In [ ]:
def simulated_annealing_fast(initial_solution: np.ndarray, max_steps: int, initial_temp: float, cooling_rate: float):
    """
    Versione veloce di SA da usare come motore di ricerca locale.
    """
    current_solution = initial_solution
    current_score = evaluate(current_solution)
    best_solution, best_score = current_solution, current_score
    temperature = initial_temp
    
    for _ in range(max_steps):
        neighbor = move(current_solution)
        neighbor_score = evaluate(neighbor)
        
        # Accetta sempre miglioramenti o mosse peggiorative con una certa probabilità
        if neighbor_score >= current_score or rng.random() < np.exp((neighbor_score - current_score) / temperature):
            current_solution, current_score = neighbor, neighbor_score
        
        if current_score > best_score:
            best_solution, best_score = current_solution, current_score
            
        temperature *= cooling_rate

    return best_solution, best_score

In [ ]:
def memetic_algorithm(generations: int, mu: int, lambda_: int):
    population = [create_random_valid_solution() for _ in range(mu)]
    #for problem 3
    #population = [np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool) for _ in range(mu)]
    best_solution_so_far, best_score_so_far = None, -1
    for gen in tqdm(range(generations)):
        offspring = []
        for _ in range(lambda_):
            parent1 = population[rng.integers(0, mu)]
            parent2 = population[rng.integers(0, mu)]
            child = crossover(parent1, parent2)
            child = move(child)
            
            improved_child, _ = simulated_annealing_fast(child, max_steps=75, initial_temp=10.0, cooling_rate=0.99)
            offspring.append(improved_child)

        combined_population = population + offspring
        scores = [evaluate(ind) for ind in combined_population]
        sorted_indices = np.argsort(scores)[::-1]
        population = [combined_population[i] for i in sorted_indices[:mu]]
        
        current_best_score = scores[sorted_indices[0]]
        if current_best_score > best_score_so_far:
            best_score_so_far = current_best_score
            best_solution_so_far = population[0]
            print(f"  Generazione {gen+1}: Nuovo record! Valore = {best_score_so_far}")

    return best_solution_so_far, best_score_so_far

In [ ]:

GENERATIONS = 50
POPULATION_SIZE = 20  # mu (dimensione della popolazione)
OFFSPRING_SIZE = 200 # lambda (numero di figli per generazione)

best_solution, best_score = memetic_algorithm(
    generations=GENERATIONS, 
    mu=POPULATION_SIZE, 
    lambda_=OFFSPRING_SIZE
)

if best_score <= 0:
    print("Nessuna soluzione trovata trovata con oggetti inseriti.")
else:
    print("Valore:", best_score)

a


 10%|█         | 1/10 [00:05<00:46,  5.18s/it]

  Generazione 1: Nuovo record! Valore = 40115.0


 20%|██        | 2/10 [00:11<00:44,  5.59s/it]

  Generazione 2: Nuovo record! Valore = 82338.0


...

  Generazione 8: Nuovo record! Valore = 305193.0


 90%|█████████ | 9/10 [00:57<00:06,  6.63s/it]

  Generazione 9: Nuovo record! Valore = 343441.0


100%|██████████| 10/10 [01:04<00:00,  6.41s/it]

  Generazione 10: Nuovo record! Valore = 381463.0
Valore: 381463.0


In [52]:

current_solution = create_random_valid_solution()
current_score = evaluate(current_solution)
_, current_score = simulated_annealing_fast(current_solution, 5000, 10, 0.99)
    

print(current_score)

806990.0
